In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.vehicle'):
    shutil.rmtree('./file/4.vehicle')

os.makedirs('./file/4.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Vehicle Page'] = input_['Vehicle Page'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                df_temp, h3_last, page = pd.DataFrame(), '', 1
                while page <= input_.loc[a, 'Vehicle Page']:
                    b = 0
                    while True:
                        b += 1

                        try:
                            resp = requests.get(f'''{input_.loc[a, 'Url']}?page={page}#applications''',
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                                
                            if resp.status_code == 200:
                                soup = BeautifulSoup(resp.text, 'lxml')
                                html = etree.HTML(str(soup))
                                
                                if html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip() == input_.loc[a, 'Part Number']:
                                    break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
            
                    # = = = = = = = = = = = = = = =
            
                    list_h3 = [h3.strip() for h3 in html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]/text()')]

                    # = = = = = = = = = = = = = = =
                    
                    list_table = html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')
            
                    # = = = = = = = = = = = = = = =

                    if not list_table:
                        raise
                        
                    # = = = = = = = = = = = = = = =

                    df_temp_temp = pd.DataFrame()
                    if len(list_h3) == len(list_table):
                        for i, h3, table in zip(range(len(list_h3)), list_h3, list_table):
                            list_th = [th.xpath('./text()')[0].strip() if th.xpath('./text()') else '' for th in table.xpath('./thead/tr/th')]
                            while '' in list_th:
                                list_th.remove('')
                
                            list_tr = table.xpath('./tbody/tr')
                            for j, tr in zip(range(len(list_tr)), list_tr):
                                list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]
                
                                df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Url'], page, i+1, h3, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')
                    elif len(list_h3) + 1 == len(list_table):
                        list_tr = list_table[0].xpath('./tbody/tr')
                        for j, tr in zip(range(len(list_tr)), list_tr):
                            list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]
                
                            df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Url'], page, 1, h3_last, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')
                
                        if len(list_table) > 1:
                            for i, h3, table in zip(range(len(list_h3)), list_h3, list_table[1:]):
                                list_th = [th.xpath('./text()')[0].strip() if th.xpath('./text()') else '' for th in table.xpath('./thead/tr/th')]
                                while '' in list_th:
                                    list_th.remove('')
                
                                list_tr = table.xpath('./tbody/tr')
                                for j, tr in zip(range(len(list_tr)), list_tr):
                                    list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]
                
                                    df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Url'], page, i+2, h3, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')
                    else:
                        raise

                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                    # = = = = = = = = = = = = = = =

                    if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 100 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} - {input_.loc[a, 'No']} > {input_.loc[a, 'Part Number']} > Page {i+1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
            
                    h3_last = list_h3[-1]

                    page += 1
                        
                # = = = = = = = = = = = = = = =

                if df_temp.empty:
                    raise

                # = = = = = = = = = = = = = = =

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(10):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Page', 'Make Model Engine No', 'Row'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.vehicle/{file}',
              f'''./file/4.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：3

输出ing...

爬虫存在error

总数量：4

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：2

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：3

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：8

输出ing...

爬虫存在error

总数量：6

输出ing...

爬虫存在error

总数量：4

输出ing...

爬虫存在error

总数量：37

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：1

输出ing...

爬虫存在error

总数量：2

输出ing...

爬虫存在error



Progress: 0it [00:00, ?it/s]

Done ~
